 <pre>
 ____   ____    _    ____  _____                          _      _     
|  _ \ / ___|  / \  / ___|| ____|     _ __ ___   ___   __| | ___| |___ 
| | | | |     / _ \ \___ \|  _| _____| '_ ` _ \ / _ \ / _` |/ _ \ / __|
| |_| | |___ / ___ \ ___) | |__|_____| | | | | | (_) | (_| |  __/ \__ \
|____/ \____/_/   \_\____/|_____|    |_| |_| |_|\___/ \__,_|\___|_|___/
                                                                        
</pre>

# DCASE-models Notebooks
Python Notebooks for [DCASE-models](https://github.com/pzinemanas/DCASE-models)

---

### About 
This Notebook reproduces the results for **Sound Event Detection (SED)** presented in:
<ul>
<li><a href="http://www.justinsalamon.com/uploads/4/3/9/4/4394963/salamon_scaper_waspaa_2017.pdf"><strong>
    Scaper: A Library for Soundscape Synthesis and Augmentation</strong></a>
    J. Salamon, D. MacConnell, M. Cartwright, P. Li, and J. P. Bello.
    In IEEE Workshop on Applications of Signal Processing to
    Audio and Acoustics (WASPAA), New Paltz, NY, USA, Oct. 2017.
    <br>
   <a type="button" class="btn btn-default btn-xs" target="_blank" href="http://www.justinsalamon.com/uploads/4/3/9/4/4394963/salamon_scaper_waspaa_2017.pdf"> PDF </a>
   <a type="button" class="btn btn-default btn-xs" target="_blank" href="https://ieeexplore.ieee.org/document/8170052"> IEEE</a>
    </li>   
</ul>

### Overview

The paper introduces [Scaper](https://github.com/justinsalamon/scaper), an open-source library for soundscape synthesis and augmentation. To illustrate the potential of the library, the authors generate a dataset of 10,000 sound-scapes, namely [URBAN-SED](http://urbansed.weebly.com/), and use it to compare the performance of two state-of-the-art algorithms for sound event detection:
- the Convolutional Recurrent Neural Net-work (CRNN) proposed by Cakir et al. [[C-CRNN]](https://ieeexplore.ieee.org/document/7933050)
- an adaptation of the Convolutional Neural Network (CNN) proposed by Salamon and Bello [[SB-CNN]](http://ieeexplore.ieee.org/document/7829341/)

### Organization

The Notebook is organized into the following sections.
* [1. Define parameters](#DefineParameters)
* [2. Extract features](#ExtractFeatures)
* [3. Load data](#LoadData)
* [4. Initialize model](#InitModel)
* [5. Train model](#TrainModel)
* [6. Evaluate model](#EvaluateModel)

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import glob
import numpy as np
import argparse

sys.path.append('../')
from dcase_models.utils.files import load_json, mkdir_if_not_exists
from dcase_models.data.data_generator import DataGenerator
from dcase_models.data.datasets import URBAN_SED
from dcase_models.data.features import MelSpectrogram
from dcase_models.model.models import SB_CNN_SED
from dcase_models.data.scaler import Scaler

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

<a id="DefineParameters"></a>
## 1. Define parameters

In [ ]:
# features parameters
sequence_time = 1.0
sequence_hop_time = 1.0
audio_hop = 640
audio_win = 1024
n_fft = 1024
sr = 22050
mel_bands = 128

# normalizer
normalizer = 'minmax'

# train parameters
early_stopping = 10
epochs = 5
considered_improvement = 0
learning_rate = 0.001
batch_size = 64
verbose = 1
optimizer = 'Adam'

# dataset parameters
dataset_path = '../../../datasets/URBAN-SED_v2.0.0'

<a id="ExtractFeatures"></a>
## 2. Extract features

Initialize Feature Extractor and Data Generator.

In [ ]:
# Initialize Feature Extractor
feature_extractor = MelSpectrogram(sequence_time=sequence_time, 
                                   sequence_hop_time=sequence_hop_time, 
                                   audio_win=audio_win, 
                                   audio_hop=audio_hop, 
                                   n_fft=n_fft, 
                                   sr=sr, mel_bands=mel_bands)

print(feature_extractor.get_features_shape())

In [ ]:
# Initialize Data Generator as an instance of URBAN_SED
dataset = URBAN_SED(dataset_path, sequence_hop_time=sequence_hop_time)
data_generator = DataGenerator(dataset, feature_extractor)

Extract the features (if they were not extracted before).

In [ ]:
if not data_generator.check_if_features_extracted():
    data_generator.extract_features()
print('Done!')

<a id="LoadData"></a>
## 3. Load data

In [ ]:
print('Loading data... ')
data_generator.load_data()
print('Done!')

And also fit a scaler and transform the training data.

In [ ]:
fold_test = 'test'

X_train, Y_train, X_val, Y_val = data_generator.get_data_for_training(fold_test, 
                                                                      evaluation_mode='train-validate-test')
scaler = Scaler(normalizer=normalizer)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [ ]:
print(X_train.shape, Y_train.shape, X_val[0].shape, Y_val[0].shape)

<a id="InitModel"></a>
## 4. Initialize model

In [ ]:
n_frames_cnn = X_train.shape[1]
n_freq_cnn = X_train.shape[2]
n_classes = Y_train.shape[1]
print(n_frames_cnn, n_freq_cnn, n_classes)
metrics = ['F1', 'ER']

model_container = SB_CNN_SED(model=None, model_path=None, n_classes=n_classes, 
                             n_frames_cnn=n_frames_cnn, n_freq_cnn=n_freq_cnn,
                             metrics=metrics)

model_container.model.summary()

<a id="TrainModel"></a>
## 5. Train model

In [ ]:
exp_folder = './'

train_arguments = {'early_stopping': early_stopping,
                  'epochs': epochs,
                  'considered_improvement': considered_improvement,
                  'learning_rate': learning_rate,
                  'batch_size': batch_size,
                  'verbose': verbose,
                  'optimizer': optimizer}

model_container.train(X_train, Y_train, X_val, Y_val, weights_path=exp_folder, **train_arguments)

<a id="EvaluateModel"></a>
## 6. Evaluate Model

In [ ]:
# Load best_weights
model_container.load_model_weights(exp_folder)

# Test model
X_test, Y_test = data_generator.get_data_for_testing(fold_test)
X_test = scaler.transform(X_test)
results = model_container.evaluate(X_test, Y_test)

print(results['F1'], results['ER'])